Write a custom ResNet architecture for CIFAR10 that has the following architecture:

        PrepLayer - Conv 3x3 s1, p1) >> BN >> RELU [64k]
        Layer1 -
            X = Conv 3x3 (s1, p1) >> MaxPool2D >> BN >> RELU [128k]
            R1 = ResBlock( (Conv-BN-ReLU-Conv-BN-ReLU))(X) [128k] 
            Add(X, R1)
        Layer 2 -
            Conv 3x3 [256k]
            MaxPooling2D
            BN
            ReLU
        Layer 3 -
            X = Conv 3x3 (s1, p1) >> MaxPool2D >> BN >> RELU [512k]
            R2 = ResBlock( (Conv-BN-ReLU-Conv-BN-ReLU))(X) [512k]
            Add(X, R2)
        MaxPooling with Kernel Size 4
        FC Layer 
        SoftMax
    Uses One Cycle Policy such that:
        Total Epochs = 24
        Max at Epoch = 5
        LRMIN = FIND
        LRMAX = FIND
        NO Annihilation
    Uses this transform -RandomCrop 32, 32 (after padding of 4) >> FlipLR >> Followed by CutOut(8, 8)
    Batch size = 512
    Target Accuracy: 90% (93% for late submission or double scores). 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!git clone https://github.com/vivekmse0205/py_vision.git /content/drive/MyDrive/session9/py_vision

Cloning into '/content/drive/MyDrive/session9/py_vision'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 1), reused 10 (delta 1), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [2]:
!pip install git+https://github.com/albumentations-team/albumentations.git

  Cloning https://github.com/albumentations-team/albumentations.git to /tmp/pip-req-build-t4v6qsgc
  Running command git clone -q https://github.com/albumentations-team/albumentations.git /tmp/pip-req-build-t4v6qsgc
  Created wheel for albumentations: filename=albumentations-1.0.2-cp37-none-any.whl size=98521 sha256=3836df6cde70a16823bcbb92c202545f7bbd663a94fcc15776da6d64fd14c3bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-t6_2ylra/wheels/e2/85/3e/2a40fac5cc1f43ced656603bb2fca1327b30ec7de1b1b66517
Successfully built albumentations
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [3]:
!pip install torchsummary
!pip install torch-lr-finder

In [9]:
# Setting path
%cd "/content/drive/MyDrive/session9/"

/content/drive/MyDrive/session9


In [6]:
import os
os.getcwd()

'/content/drive/MyDrive/session9/py_vision'

In [7]:
from __future__ import print_function
import torch
import torchvision
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
from py_vision.models import resnet_custom

In [11]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [13]:
import torchvision.transforms as transforms

In [14]:
dataset=torchvision.datasets.CIFAR10(root='./data/',
                                             train=True,                                            
                                             download=True,
                                             transform=transforms.ToTensor()
                                          )
data_loader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=16,
                                         shuffle=True, 
                                         num_workers=2)


Extracting ./data/cifar-10-python.tar.gz to ./data/
